### Dataset

**20 Newsgroups** ( http://qwone.com/~jason/20Newsgroups/ )

The dataset has approximately 18,000 Newsgroups documents on 20 topics split in two subsets: one for training (or development) and the other one for testing (or for performance evaluation).

For this assignment, I will use only documents from 3 categories ('talk.religion.misc','comp.graphics','sci.space').

### 1. Pre-processing

In this step, we'll download the dataset using the scikit-learn library. The sample program to import the library is given below.

This involves two steps:

    1) Fetch dataset corresponding to the three following categories:

    *  'talk.religion.misc'
    *  'comp.graphics'
    *  'sci.space'
    
    2) Remove stop words* and create count vectors for the train and test datasets. 
    

    *Stop words in this context refer to words that occur very frequently and thus contain little information about the type of the article itself.  For e.g., 'a','and','the' etc. See https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/feature_extraction/stop_words.py for the list of stop words used in scikit when 'english' is given as input.
        

In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
import numpy as np
from scipy.sparse import find
import math as mth

**Step1: ** Fetch the dataset for the three aforementioned categories using scikit-learn library.

In [4]:
categories = ['talk.religion.misc','comp.graphics','sci.space']

num_categories = len(categories)

#Loading training data

data_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

#Loading testing data

data_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

# Loading the class labels for training and testing data

y_train, y_test = data_train.target, data_test.target

In [5]:
# Total number of documents in train and test datasets

num_train = len(data_train.target)
num_test = len(data_test.target)

print("Dataset contatins \n "
       +str(num_train)+" train documents, \n "
       + str(num_test) + " test documents." )

Dataset contatins 
 1554 train documents, 
 1034 test documents.


In [6]:
print(data_train.data[0])

From: nicho@vnet.IBM.COM (Greg Stewart-Nicholls)
Subject: Re: Biosphere II
Reply-To: nicho@vnet.ibm.com
Disclaimer: This posting represents the poster's views, not those of IBM
News-Software: UReply 3.1
X-X-From: nicho@vnet.ibm.com
            <1q1kia$gg8@access.digex.net>
Lines: 18

In <1q1kia$gg8@access.digex.net> Pat writes:
>In article <19930408.043740.516@almaden.ibm.com> nicho@vnet.ibm.com writes:
>>In <1q09ud$ji0@access.digex.net> Pat writes:
>>>Why is everyone being so critical of B2?
>> Because it's bogus science, promoted as 'real' science.
>It seems to me, that it's sorta a large engineering project more
>then a science project.
  Bingo.
>B2 is not bench science,  but rather a large scale attempt to
>re-create a series of micro-ecologies.   what's so eveil about this?
 Nothing evil at all. There's no actual harm in what they're doing, only
how they represent it.

 -----------------------------------------------------------------
 .sig files are like strings ... every yo-yo's

In [43]:
print(data_train.target_names[data_train.target[0]])

sci.space


**Step2:** Remove stop words and create count vectors for the train and test datasets.

   We use the CountVectorizer method to extract features (counts for each word). Note that words from both training and testing data are needed to build the count table.

In [8]:
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit(data_train.data + data_test.data)
x_train = vectorizer.transform(data_train.data)
x_test = vectorizer.transform(data_test.data)    

In [41]:
x_train_array=x_train.toarray()

In [42]:
len(vectorizer.get_feature_names())

37830

# 2. Building the classifier

Now, let's build a Multinomial Naive Bayes classifier that takes feature vector from the test data as input and classifies as one of the three classes ('talk.religion.misc','comp.graphics','sci.space').

Complete the training function MultiNB_train() in the cell below to train a Multiomial Naive Bayes classifier that takes "x_train","y_train","alpha" as inputs and returns the likelihood probability matrix "theta" and the prior distribution  "prior" on the document category.

"prior" is a vector of length equal to num_categories where the $i$-th element is defined as
$$ prior (i) = \frac{\text{ # of train documents with category i}}{\text{Total number of train documets}} $$

"theta" ($\theta$) is the  matrix with the $(c,i)$th element defined by

 $$ \theta(c,i) = P(w_i/c) =  \frac{N_{ci} + \alpha }{N_c + |V| \alpha}$$
 
 where,
 * $P(w_i/c)$ refers to the probability of seeing the $i$th word in the vocabulary given that class type is $c$.
 * $N_{ci}$ refers to the total number of times the word  $i$ appeared in the training documents of class type $c$.
 * $N_c$ is the total number of words in the documents of type $c$
    $$N_c = \sum_{d \in T[c]} N_{cd}$$
    where, $T[c]$ refers to the documents of type $c$.
 * $|V|$ is the size of the vocabulary.
 * $\alpha$ is the laplace smoothing parameter

In [11]:
#Function for calculation total word count in a
def calculateWordCount(x_train, individual_wc_count):
    word_count = 0
    for i in range(len(x_train)):
        if(x_train[i]>0):
            individual_wc_count[i] +=x_train[i]
            word_count += x_train[i]
    return (word_count,individual_wc_count)

In [12]:
#Calculate prior probabilities of each document
def calculatePrior(y_train, num_categories):
    doc_cateories_count = [0] * num_categories
    doc_total_number = len(y_train)
    
    for i in range(len(y_train)):
        doc_cateories_count[y_train[i]]+=1
        
    prior = [0] * num_categories
    for i in range(num_categories):
        prior[i] = float(doc_cateories_count[i])/doc_total_number;
    return (prior)

In [13]:
#Function for calculating theta
def calculateTheta(individual_wc_in_cat, total_wc_in_cat, V, alpha, num_categories):
    theta = [[0 for x in range(V)] for y in range(num_categories)]
    for categor_number in range(num_categories):
        for word_number in range(V):
            theta[categor_number][word_number] = float(individual_wc_in_cat[categor_number][word_number]+alpha)/(total_wc_in_cat[categor_number]+(alpha*V))
    
    return(theta)

In [14]:
def MultiNB_train(x_train,y_train, num_categories, alpha):
    V = len(vectorizer.get_feature_names()) # Calcualte Size of vocabulary
    x_train_array = x_train.toarray() # convert transformed data in two-d array
    
    total_wc_in_cat = [0] * num_categories
    individual_wc_in_cat = [[0 for x in range(V)] for y in range(num_categories)] 
    prior = calculatePrior(y_train, num_categories)
    for i in range(len(x_train_array)):
        wc_count,individual_wc_in_cat[y_train[i]] = calculateWordCount(x_train_array[i], individual_wc_in_cat[y_train[i]])
        total_wc_in_cat[y_train[i]]+=wc_count
    theta = calculateTheta(individual_wc_in_cat, total_wc_in_cat, V, alpha, num_categories)
    return(theta, prior)

Now, let us train the model to learn the likelihood parameters $\theta$

In [69]:
theta,prior = MultiNB_train(x_train,y_train, num_categories, alpha = 10)

Multinomial Naive Bayes Classifier implementation

In [36]:
def MultiNB_classify(x_test_sample, theta, prior,num_categories):
    set_prob=True
    max_prob = 0
    pred_class = 0
    x_test_sample_array = x_test_sample.toarray()[0]
    for category in range(num_categories):
        cat_prob_input = mth.log10(prior[category])
        for word in range(len(x_test_sample_array)):
            if(x_test_sample_array[word]>0):
                cat_prob_input += (mth.log10(theta[category][word]) * x_test_sample_array[word])
        if(set_prob == True):
            max_prob = cat_prob_input
            set_prob = False
        if(cat_prob_input > max_prob):
            max_prob = cat_prob_input
            pred_class = category 
    return  pred_class

   

Let us test our classifier on the first sample of testing dataset.

In [49]:
pred_class = MultiNB_classify(x_test.getrow(0),theta, prior,num_categories)

print("predicted class:" + str(pred_class))
print("actual class:" + str(y_test[0]))

predicted class:0
actual class:0


# 3. Evaluating the classifier

The following code below runs your classifier on every data sample from the testing dataset and stored them in "y_pred".

In [65]:
y_pred = []
for i in range(num_test):
    pred_class = MultiNB_classify(x_test.getrow(i),theta, prior,num_categories)
    y_pred.append(pred_class)

The following cell evaluates your result by comparing it with the test labels.

In [66]:
score = metrics.accuracy_score(y_test,y_pred)
print(score)
print("accuracy: %0.3f" % score)
print(metrics.classification_report(y_test,y_pred))

0.959381044487
accuracy: 0.959
             precision    recall  f1-score   support

          0       0.96      0.96      0.96       389
          1       0.95      0.97      0.96       394
          2       0.98      0.94      0.96       251

avg / total       0.96      0.96      0.96      1034



Find the classification error (1-score) over the test set for various values of the smoothing parameter α and by trial and error find a good value of α.

In [68]:
alpha = 0.01
max_score = -1
while 1:
    theta,prior = MultiNB_train(x_train,y_train, num_categories, alpha)
    y_pred = []
    for i in range(num_test):
        pred_class = MultiNB_classify(x_test.getrow(i),theta, prior,num_categories)
        y_pred.append(pred_class)
    score = metrics.accuracy_score(y_test,y_pred)
    if(max_score < score):
        max_score = score
        alpha=alpha+1
    elif(max_score == -1):
        max_score = score
        alpha=alpha+1
    else:
        break
max_score = -1
if(alpha < 1):
    alpha2 = 0.1
elif(alpha<2):
    alpha2 = alpha - 1
else:
    alpha2 = alpha - 2
    
while alpha2 <= alpha:
    theta,prior = MultiNB_train(x_train,y_train, num_categories, alpha2)
    y_pred = []
    for i in range(num_test):
        pred_class = MultiNB_classify(x_test.getrow(i),theta, prior,num_categories)
        y_pred.append(pred_class)
    score = metrics.accuracy_score(y_test,y_pred)
    print(alpha2, score)
    if(max_score <= score):
        max_score = score
        alpha2+=0.1
    elif(max_score == -1):
        max_score = score
        alpha2+=0.1
    else:
        break
        
print(alpha2-0.1,max_score,score)

(1.0099999999999998, 0.95647969052224369)
(1.1099999999999999, 0.95551257253384914)
(1.0099999999999998, 0.95647969052224369, 0.95551257253384914)
